<a href="https://colab.research.google.com/github/OlegPozovnoy/MailRuMLHomeworks/blob/master/HW3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/datasets/')

Drive already mounted at /datasets/; to attempt to forcibly remount, call drive.mount("/datasets/", force_remount=True).


In [15]:
!ls "/datasets/My Drive/datasets/"

players.pkl  results.pkl  tournaments.pkl


In [0]:
import pickle
import pandas as pd

In [0]:
tournaments = pd.read_pickle("/datasets/My Drive/datasets/tournaments.pkl")
#tournaments = pd.read_pickle("./tournaments.pkl")

tournaments_cleaned = [(v['id'], v['name'], v['dateStart'][0:4], v['type']['name'], v['questionQty']) for k,v in tournaments.items() if v['questionQty'] != None]
del tournaments

Создадим трейн и тест набор турниров.

In [18]:
train_id = [ v[0] for v in tournaments_cleaned if int(v[2])<= 2019]
test_id = [v[0] for v in tournaments_cleaned if int(v[2])== 2020]

print(len(train_id))
print(len(test_id))

3921
418


In [0]:
results = pd.read_pickle("/datasets/My Drive/datasets/results.pkl")
#results = pd.read_pickle("./results.pkl")
results_cleaned = {k: v for k,v in results.items() if k in train_id or k in test_id}

results_cleaned2 = {}
for k,v in results_cleaned.items():
  results_cleaned2[k] = []
  for s in v:
    if s.get('mask') != None:
      results_cleaned2[k].append((s.get('mask'),s.get('questionsTotal'), [k.get('player').get('id') for k in s.get('teamMembers')]))

del results
del results_cleaned

In [20]:
train_id = [id for id in train_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]
test_id = [id for id in test_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]

print(len(train_id))
print(len(test_id))

3486
173


In [65]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import tqdm
import gc

all_words = []

for k,v in results_cleaned2.items():
  for team in v: 
    for p in range(len(team[0])):
        all_words.append(str(k)+'_'+str(p))
    for m in team[2]:
        all_words.append(m)

all_words = set(all_words)
print(len(all_words))

all_results = {}
for s in all_words:
  all_results[s] = [0,0] # total rating and num of competetions

print (len(results_cleaned2))
vocabulary =  set()

logistic = LogisticRegression(fit_intercept = False)

def process_tournament(id):
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]) )
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no
  word_to_idx = dict([(y,x) for x,y in enumerate(words)])
  idx_to_word = dict([(x,y) for x,y in enumerate(words)])       
  
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<=2 categories')
    return

  print(X.shape)
  logistic.fit(X, Y)

  
  print(logistic.predict_proba(X))
  #print(len(logistic.coef_[0]))

  for s in range(len(logistic.coef_[0])):
    if isinstance(idx_to_word[s], int): ## tean no
        #print('team '+ str(idx_to_word[s]))
        for player in results_cleaned2[id][idx_to_word[s]][2]:
            #print(player)
            all_results[player][1] +=1
            all_results[player][0] +=logistic.coef_[0][s]
    else: #question no
        #print('question '+ str(idx_to_word[s]))
        all_results[idx_to_word[s]][1] +=1
        all_results[idx_to_word[s]][0] +=logistic.coef_[0][s]

#print(process_tournament(1521))
process_tournament(1526)

357955
4339
(46746, 1155)
[[0.12146259 0.87853741]
 [0.0849564  0.9150436 ]
 [0.02114813 0.97885187]
 ...
 [0.94584594 0.05415406]
 [0.99235288 0.00764712]
 [0.93338122 0.06661878]]


Пройдемся по всем турнирам и посчитаем рейтинг всех игроков и вопросов в заданном турнире, за неимением лучшего, усредним по каждому игроку рейтинг в турнире

In [22]:
import tqdm

for tourn in tqdm.tqdm(train_id):
  print(tourn)
  process_tournament(tourn)




  0%|          | 0/3486 [00:00<?, ?it/s]

  0%|          | 3/3486 [00:00<02:17, 25.41it/s]

22
(3960, 134)
134
76
(4590, 141)
141
141
(4320, 138)
138
226
(5130, 147)
147
315
(6570, 163)




  0%|          | 5/3486 [00:00<02:36, 22.28it/s]

  0%|          | 7/3486 [00:00<03:00, 19.29it/s]

163
484
(8190, 181)
181
583
(6390, 161)
161
652
(1710, 83)
83
1082
(3690, 131)




  0%|          | 10/3486 [00:00<02:42, 21.43it/s]

  0%|          | 12/3486 [00:00<03:10, 18.20it/s]

...

(115, 28)
28
6135
(1440, 78)
78
6137
(3636, 137)
137
6138
(1692, 83)
83
6141
(900, 61)
61
6144
(1296, 72)
72
6145
(1080, 69)
69
6146
(396, 47)
47
6147
(720, 63)
63
6149
<=2 categories
6150




100%|█████████▉| 3479/3486 [14:55<00:00, 26.82it/s]

(2160, 192)
192
6151
(612, 53)
53
6159
(1188, 69)
69
6161
(1035, 68)
68
6170
(768, 64)
64
6173
(960, 76)
76
6183
(2448, 104)
104
6191
(792, 58)
58
6249
(75384, 1050)
1050
6254
(121968, 1061)
1061
6255
(213516, 2235)




100%|█████████▉| 3479/3486 [15:12<00:00, 26.82it/s]

100%|██████████| 3486/3486 [15:43<00:00,  3.69it/s]

2235
6349
(900, 61)
61


In [53]:
ranking = [(k, 0 if v[1]== 0 else v[0]/v[1],v[1]) for k,v in all_results.items()]
print(ranking)

ranking.sort(key = lambda x: -x[1])

print(ranking)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [55]:
import statistics as stat

def is_player(t):
  return isinstance(t, int)
  
players_ranking = [ x for x in ranking if is_player(x[0]) ]
blank_rank = stat.mean ([ x[1] for x in ranking if is_player(x[0])])
print(blank_rank)

def get_team_ranking(players):
  result = []
  print('Team:',players)
  if (len(players)==0):
    return blank_rank
  for player in players:
    if (all_results[player][1]>0): # num of competetions
      print(player, ' rating is ', all_results[player][0])
      result.append(all_results[player][0])
    else:
      result.append(blank_rank)
  return stat.mean(result)


-0.8203929165969827


In [60]:
spearman = []
pearson = []

test_id = [x for x in test_id if len(results_cleaned2[x])>1] ## фильтруем турниры с 1 командой

for tourn_id in test_id:
  print('Tournament:', tourn_id)
  rating = []
  ranking = []
  i=1
  for team in results_cleaned2[tourn_id]:
    rating.append(get_team_ranking(team[2]))
    ranking.append(i)
    i+=1
  print ('Spearman: ',stats.spearmanr(rating, ranking))
  print ('Pearson: ',stats.pearsonr(rating,ranking))
  spearman.append(stats.spearmanr(rating, ranking)[0])
  pearson.append(stats.pearsonr(rating,ranking)[0])



Streaming output truncated to the last 5000 lines.
4064  rating is  127.08281904494652
16332  rating is  555.9338056969818
26818  rating is  371.9966667970877
36035  rating is  313.19198793067056
70750  rating is  308.24586644388916
Team: [2421, 6482, 13782, 25882, 32458, 34846]
2421  rating is  322.5948766852874
6482  rating is  721.8713264342828
13782  rating is  550.4690524890223
25882  rating is  386.0495352921078
32458  rating is  231.60213568980998
34846  rating is  361.7483899936098
Team: [62804, 62805, 62806, 90870, 103180, 122817]
62804  rating is  33.98711677513482
62805  rating is  31.658430061936638
62806  rating is  34.31578293793603
90870  rating is  -31.008840274955954
103180  rating is  -37.28958207838904
122817  rating is  -85.57587487770621
Team: [53361, 94935, 111545, 111553, 111657, 123644, 138615, 165774]
53361  rating is  -4.117798077356909
94935  rating is  -2.048968307740333
111545  rating is  33.46240687522801
111553  rating is  37.06462856495704
111657  rating

AttributeError: ignored

In [61]:
print(spearman)
print(pearson)
print(stat.mean(spearman))
print(stat.mean(pearson))

[-0.6693432794414562, -0.7241578859097109, -0.7410142238713666, -0.459470763861597, -0.15158871108345726, -0.46098163858462515, -0.15935278663472535, -0.33406554651485, -0.37859389709240016, -0.20329301075268819, -0.36125945062374787, -0.7887846560143191, -0.47354201171529736, -0.4177167468341766, -0.5612443751550156, -0.41141134790425, -0.7058823529411765, -0.6269337312310432, -0.6502935661024811, -0.6534933973589436, -0.8153612703708899, -0.6999097512656836, -0.6564424362858806, -0.7649204920492048, -0.48842912287855955, -0.41323501670240653, -0.3168899913898024, -0.23043478260869563, -0.6695107821288951, -0.7804854070660521, -0.8555140262036813, -0.785872709017069, -0.07268171777893932, -0.8215654077723044, -0.6736842105263157, -0.7741076075632202, 0.12077922077922078, -0.5191183632957764, -0.4656069963405541, -0.5504093129093128, -0.7890243902439027, -0.6131994261119081, -0.8391608391608393, -0.9102353585112206, -0.7058062212701387, -0.43715728715728724, -0.7649258259577761, -0.753